In [ ]:
import pandas as pd

from bs4 import BeautifulSoup
import requests

In [ ]:
baselink = "https://www.aekszerek.hu/arany/noi-arany-nyaklancok.html?lis="
n_of_pages=4
urls_for_items = [] # e.g."https://www.aekszerek.hu/ezust/nyaklancok/noi/ezust-nyaklanc-sziv-p156367.html",

for page in range(n_of_pages):
    source = requests.get(baselink + str(page)).content
    soupbase = BeautifulSoup(source, "html.parser")

    item_imgs_on_this_page = soupbase.find_all('div', class_="item_img")
    urls_for_items_on_this_page = ["https://www.aekszerek.hu/" + item.a['href'] for item in item_imgs_on_this_page]

    urls_for_items = urls_for_items + urls_for_items_on_this_page

print(f"{len(urls_for_items)} item-url is found after going through {n_of_pages} pages")

In [ ]:
def get_price_for_item(soup):
    pricevalue_in_string = soup.find('span', id="pricevalue").text # '\n56 510 Ft \n'
    numbers_in_pricevalue = [s for s in pricevalue_in_string.split() if s.isdigit()] # [56, 510]
    price_in_int = int(''.join((numbers_in_pricevalue)))
    return price_in_int

def get_itemdetails_for_item(soup):
    item_details_with_noise = soup.find('div', class_='col-md-8 col-sm-12 col-xs-12 margin-bottom10 toggle-content')
    item_details_in_string = item_details_with_noise.find('div', class_='margin-bottom10').text

    details = ["Anyag", "Nyaklánc kapocs", "Motívum", "Célcsoport", "Ékszer súlya", "Tisztaság", "Arany színe", "Hossz", "Szélesség", "Magasság", "Kő"]
    item_details_dict = {}
    for detail in details:
        try:
            value = item_details_in_string.split(f'\n{detail}: ')[1].split(',\n')[0]
            item_details_dict[detail] = value
        except:
            item_details_dict[detail] = None

    return item_details_dict

In [ ]:
# 2p alatt 65ig
df = pd.DataFrame()

for i, itemurl in enumerate(urls_for_items):
    if i%10 == 0:
        print(f"{i} out of {len(urls_for_items)}")
    soup = BeautifulSoup(requests.get(itemurl).content, "html.parser")
    item_title = soup.title.text
    price = get_price_for_item(soup)
    itemdetails = get_itemdetails_for_item(soup)
    
    # df = df.append(pd.Series(dict({'price': price, 'url': itemurl, 'title':item_title, **itemdetails})), ignore_index=True)
    df = pd.concat([df, pd.Series(dict({'price': price, 'url': itemurl, 'title':item_title, **itemdetails}))], axis=1)
df = df.T.reset_index(drop=True)

In [ ]:
df.to_csv('nyaklanc.csv', index=False)